# Heart Rate Prediction Starter Kit for Colab

This notebook provides a starter kit for running the Heart Rate Prediction project in Google Colab. It demonstrates how to import the necessary functions, and run the complete pipeline.

## 1. Setup

This notebook assumes you have already cloned the repository and have the required dependencies installed.

In [ ]:
!git clone https://github.com/AndrewBlur/HeartRatePrediction.git

## 2. Download the Data

The Dataset comes from [MEDVSE repository](https://github.com/MahdiFarvardin/MEDVSE)

## 3. Import Functions and Run the Pipeline

Now we can import the functions from the `src` directory and run the pipeline, just like in `main.py`.

In [ ]:
import os
os.chdir('/content/HeartRatePrediction')

In [ ]:
from HeartRatePrediction.src.data_loader import load_mths_data
from HeartRatePrediction.src.preprocessing import process_data
from HeartRatePrediction.src.eda import inspect_data, perform_eda
from HeartRatePrediction.src.models import build_cnn_model, build_lstm_model, build_rf_model
from HeartRatePrediction.src.training import train_cnn_model, train_lstm_model, train_rf_model
from HeartRatePrediction.src.evaluation import evaluate_models
from HeartRatePrediction.src.prediction import HeartRateMonitor
from sklearn.model_selection import train_test_split
import os

In [ ]:
DATA_PATH = './dataset'

# --- Data Loading ---
raw_data = load_mths_data(DATA_PATH)
print("Data Loading Complete.")

# --- Data Inspection ---
inspect_data(raw_data)

# --- Data Processing ---
X_processed, y_processed = process_data(raw_data)
print(f"Processed X Shape: {X_processed.shape}")
print(f"Processed Y Shape: {y_processed.shape}")

# --- Exploratory Data Analysis ---
perform_eda(X_processed, y_processed)

# --- Train-Test Split ---
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, test_size=0.2, random_state=42, shuffle=True
)
print(f"Training Data: {X_train.shape}")
print(f"Testing Data: {X_test.shape}")

# --- Model Training ---
cnn_model = build_cnn_model((30, 3))
lstm_model = build_lstm_model((30, 3))
rf_model = build_rf_model()

cnn_model, _ = train_cnn_model(cnn_model, X_train, y_train, X_test, y_test)
lstm_model, _ = train_lstm_model(lstm_model, X_train, y_train, X_test, y_test)
rf_model = train_rf_model(rf_model, X_train, y_train)

models = {'cnn': cnn_model, 'lstm': lstm_model, 'rf': rf_model}

# --- Model Evaluation ---
evaluate_models(models, X_test, y_test)

# --- Prediction Example ---
video_path = './HeartRatePrediction/sample_videos/test_video.mp4'
if not os.path.exists(video_path):
    print(f"'{video_path}' not found. Please upload a video for prediction.")
else:
    app = HeartRateMonitor(model=rf_model, model_type='rf', fs=30)
    hr, spo2 = app.predict(video_path)
    print(f"Predicted Heart Rate: {hr}")
    print(f"Predicted SpO2: {spo2}")